In [1]:
% matplotlib inline 

from __future__ import print_function

import nibabel as nib
from nilearn.image import resample_img

import matplotlib.pyplot as plt

import numpy as np
import os
import os.path

# The following are a progress bar, these are not strictly necessary:
from ipywidgets import FloatProgress
from IPython.display import display

In [2]:
percentiles = range(10)

# unthresholded z-maps from neurosynth:
zmaps = [os.path.join(os.getcwd(), 'ROIs_Mask', fname) for fname in os.listdir(os.path.join(os.getcwd(), 'ROIs_Mask'))
         if 'z.nii.gz' in fname]

# individual, binned gradient maps, in a list of lists:
gradmaps = [[os.path.join(os.getcwd(), 'data', 'Outputs', 'Bins', str(percentile), fname)
             for fname in os.listdir(os.path.join(os.getcwd(), 'data', 'Outputs', 'Bins', str(percentile)))]
            for percentile in percentiles]

# a brain mask file:
brainmask = os.path.join(os.getcwd(), 'ROIs_Mask', 'rbgmask.nii')

In [3]:
def zinsidemask(zmap, mask):
    # check if mapping is the same, otherwise transform:
    if zmap.shape != mask.shape:
        zmap = resample_img(zmap, target_affine=mask.affine, target_shape=mask.shape)
    zaverage = zmap.dataobj[np.not_equal(mask.dataobj, 0)].mean()
    return zaverage

In [ ]:
zmap = nib.load(zmaps[0])

zaverages = np.zeros([len(zmaps), len(gradmaps), len(gradmaps[0])])

progbar = FloatProgress(min=0, max=zaverages.size)
display(progbar)
for i1, zmapfile in enumerate(zmaps):
    zmap = nib.load(zmapfile)
    for i2, percentile in enumerate(percentiles):
        for i3, gradmapfile in enumerate(gradmaps[percentile]):
            gradmap = nib.load(gradmapfile)
            zaverages[i1, i2, i3] = zinsidemask(zmap, gradmap)
            progbar.value += 1
